In [3]:
# Retrieving bill text session by session

%load_ext sql
%sql postgresql://work@localhost/postgres


bills = %sql select S.session_id, B.number_only, T.text_en from bills_bill B inner join bills_billinsession S on S.bill_id = B.id inner join bills_billtext T on T.bill_id = B.id where B.number_only < 200 and B.institution = 'C' order by S.session_id, number_only

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
660 rows affected.


In [4]:
# Removing duplicate bills from query

class billData:
    def __init__(self, session, num, content):
        self.session = session
        self.number = num
        self.text = content

output = []

last_bill = ""
last_session= ""
for bill in bills:
    if (last_bill != bill.number_only):
            output.append(billData(bill.session_id, bill.number_only, bill.text_en))
    last_bill = bill.number_only
print (len(output))

473


In [5]:
# output files to each directory
import os
def assure_path_exists(path):
        dir = os.path.dirname(path)
        if not os.path.exists(dir):
                os.makedirs(dir)

for bill in output:
    path = './data/'+bill.session+'/{0:03d}.txt'.format(bill.number)
    assure_path_exists(path)
    fp = open(path, 'w')
    fp.write(bill.text)
    #print("Session: "+bill.session+" Bill: "+ '{0:03d}'.format(bill.number)) 


In [6]:
# extract hansard counts for word "omnibus"
%load_ext sql
%sql postgresql://work@localhost/postgres

hansard_counts = %sql select S.session_id, B.number_only, count(B.id) from bills_bill B inner join bills_billinsession S on S.bill_id = B.id inner join hansards_statement_bills HB on HB.bill_id = B.id inner join hansards_statement HS on HS.id = HB.statement_id where B.institution = 'C' and B.number_only < 200 and HS.content_en like '%omnibus%' group by B.number_only, S.session_id order by S.session_id, B.number_only


The sql extension is already loaded. To reload it, use:
  %reload_ext sql
248 rows affected.


In [9]:
# write list of identified omnibus bills to omnibus.csv file
for count in hansard_counts:
    if count.count >= 10:
        path = './data/'+count.session_id+'/omnibus.csv'
        assure_path_exists(path)
        fp = open(path, 'a')
        fp.write(str(count.number_only)+'\n')
    #print("Session: "+bill.session+" Bill: "+ '{0:03d}'.format(bill.number)) 
